# Depth Capture into Voxels

- Download the dataset
- Generate the voxel map.
- Visualize the map

## Configuration

In [1]:
import rosbag
import numpy as np
from scipy.spatial.transform import Rotation as R
import open3d as o3d

import matplotlib.pyplot as plt
import math

In [2]:
# --- CONFIG ---
bag_path = "bags/indoor_flying1_gt.bag"
depth_topic = "/davis/left/depth_image_rect"
pose_topic  = "/davis/left/pose"
info_topic  = "/davis/left/camera_info"

depth_frames = []
poses_frame_Twc = []

# INFLATE: Camera rig inflation config
CAMERA_RIG_DIAMETER = 0.40  # 40cm (typical small drone/robot)
SAFETY_MARGIN = 0.10        # 10cm extra safety
INFLATION_RADIUS = (CAMERA_RIG_DIAMETER / 2) + SAFETY_MARGIN  # 30cm total
VOXEL_SIZE = 0.05  # 5cm voxels

# Read the rosbag file

In [3]:
def read_bag_file(bag_path, info_topic, pose_topic, depth_topic ):

    # --- OPEN BAG ---
    with rosbag.Bag(bag_path, "r") as bag:
        
        print("Reading  camera intrinsics...")
        cam_info = next(bag.read_messages(topics=[info_topic]))[1]
        
        K = np.array(cam_info.K).reshape(3, 3)
        fx, fy = K[0, 0], K[1, 1]
        cx, cy = K[0, 2], K[1, 2]
        print(f"Camera intrinsics: fx={fx:.3f}, fy={fy:.3f}, cx={cx:.3f}, cy={cy:.3f}")

        print("Reading poses...")
        for topic, msg, t in bag.read_messages(topics=[pose_topic]):
            ts = msg.header.stamp.to_sec()

            if hasattr(msg, "pose"):
                pose = msg.pose.pose if hasattr(msg.pose, "pose") else msg.pose
            else:
                pose = msg

            q = pose.orientation
            r = R.from_quat([q.x, q.y, q.z, q.w]).as_matrix()
            p = np.array([pose.position.x, pose.position.y, pose.position.z])

            # Homogeneous transform (world <- camera)
            Twc = np.eye(4)
            Twc[:3, :3] = r
            Twc[:3, 3] = p

            poses_frame_Twc.append((ts, Twc))

        print(f"Loaded {len(poses_frame_Twc)} poses.")

        print("Reading depth frames...")
        for topic, msg, t in bag.read_messages(topics=[depth_topic]):
            ts = msg.header.stamp.to_sec()
            
            # Convert bytes to numpy array
            img_np = np.frombuffer(msg.data, dtype=np.float32)
            
            # Reshape to 2D image
            img_np = img_np.reshape((msg.height, msg.width))

            depth_frames.append((ts, img_np))

        print(f"Loaded {len(depth_frames)} depth frames.")

        return cam_info, poses_frame_Twc, depth_frames



## Match the timestamps between depth frame and pose data

In [4]:
def match_depth_pose(depth_frames, poses_Twc, max_dt=0.05):
    """ max_dt = threshold for checking pose and depth has same timestamp """
    paired_depths = []
    paired_poses = []
    pose_ts = np.array([ts for ts, _ in poses_Twc])

    for dts, depth in depth_frames:
        idx = np.argmin(np.abs(pose_ts - dts))
        if abs(pose_ts[idx] - dts) <= max_dt:
            paired_depths.append(depth)
            paired_poses.append(poses_Twc[idx][1])
    print(f"Matched {len(paired_depths)} depth frames with poses.")
    return paired_depths, paired_poses

## Generate Point Cloud

In [5]:
def depth_to_world_points(depth, Twc, fx, fy, cx, cy, stride=4):
    """Back-project a depth map to world coordinates """
    H, W = depth.shape
    # reduce memory and computation
    vs, us = np.mgrid[0:H:stride, 0:W:stride]
    z = depth[::stride, ::stride]

    # filter out invalid depth
    valid = np.isfinite(z) & (z > 0)
    if not np.any(valid):
        return np.zeros((0, 3))
    
    us = us[valid]
    vs = vs[valid]
    z = z[valid]

    # pinhole camera model
    x_c = (us - cx) * z / fx
    y_c = (vs - cy) * z / fy
    pts_c = np.stack([x_c, y_c, z, np.ones_like(z)], axis=1).T  # 4xN

    # transform to world
    pts_w = (Twc @ pts_c)[:3, :].T  
    return pts_w


def generate_point_cloud(pcd, depth_images, poses_Twc, K):
    # read camera intrinsic
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]

    for i, (depth, Twc) in enumerate(zip(depth_images, poses_Twc)):
        
        pts = depth_to_world_points(depth, Twc, fx, fy, cx, cy, stride=4)
        if pts.size == 0:
            continue

        # append new points to running map
        if len(pcd.points) == 0:
            pcd.points = o3d.utility.Vector3dVector(pts)
        else:
            old = np.asarray(pcd.points)
            pcd.points = o3d.utility.Vector3dVector(np.vstack([old, pts]))

    assert len(pcd.points) > 0


def apply_colorizaation(pcd):
    pts = np.asarray(pcd.points)
    norm_y = (pts[:, 1] - pts[:, 1].min()) / (np.ptp(pts[:, 1]) + 1e-8)
    colors = plt.cm.viridis(norm_y)[:, :3]
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pts


def create_voxel_map(pcd, voxel_size = 0.01):
    vg = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)

def save_voxel_map(vg, file_path="map_voxels.ply"):
    o3d.io.write_voxel_grid(file_path, vg)

# Inflate the voxel grid

In [6]:
def points_to_voxel_indices(points, voxel_size, origin=None):
    """
    Convert Nx3 points -> unique integer voxel indices (M,3) and return origin used.
    """
    # handling empty input -> prevents erros from min()
    if points.shape[0] == 0:
        return np.zeros((0,3), dtype=int), np.zeros(3)
    if origin is None:
        # finds minimum point of points x,y,z
        min_p = points.min(axis=0)
        # Put our grid to start slightly before the smallest point
        # -> even the minimum point falls inside the first voxel.
        origin = min_p - voxel_size * 1e-3 # epsilon: voxel_size * 1e-3
        print(f"[min_p] {min_p}  |  [origin] {origin}")
    idx = np.floor((points - origin) / voxel_size).astype(np.int64)
    
    uniq_idx = np.unique(idx, axis=0) # remove duplicates

    print(f"idx: {idx}\n uniq_idx: {uniq_idx}")
    return uniq_idx, origin

def voxel_indices_to_centers(indices, voxel_size, origin):
    """
    Convert integer voxel indices (M,3) -> centers (M,3) world coordinates
    """
    centers = origin + (indices.astype(np.float64) + 0.5) * voxel_size
    return centers

def inflate_voxel_indices(indices, voxel_size, radius_m):
    """
    Given integer indices array (M,3), inflate each voxel by radius_m (meters).
    Returns new unique indices array.
    """
    if radius_m <= 0 or len(indices) == 0:
        return indices
    
    # radius in voxels
    k = int(math.ceil(radius_m / voxel_size)) # 0.3 / 0.05 = 6
    
    # precompute offsets within euclidean radius
    offsets = []
    r2 = (radius_m / voxel_size) ** 2
    for dx in range(-k, k+1):
        for dy in range(-k, k+1):
            for dz in range(-k, k+1):
                if dx*dx + dy*dy + dz*dz <= r2:
                    offsets.append((dx, dy, dz))
    offsets = np.array(offsets, dtype=np.int64)  # P x 3
    
    print(f"Inflation: {len(offsets)} offset voxels in sphere of radius {k} voxels")
    
    # apply offsets (broadcast)
    expanded = (indices[:, None, :] + offsets[None, :, :]).reshape(-1, 3)
    uniq = np.unique(expanded, axis=0)
    return uniq

def build_voxel_set_from_points(all_points, voxel_size=0.05, origin=None, inflate_radius=0.0):
    """
    all_points: Nx3 array
    returns: voxel_indices (M,3) unique, origin
    """
    vox_idx, origin = points_to_voxel_indices(all_points, voxel_size, origin)
    
    original_count = len(vox_idx)
    
    if inflate_radius > 0 and len(vox_idx) > 0:
        print(f"Original voxels: {original_count}")
        vox_idx = inflate_voxel_indices(vox_idx, voxel_size, inflate_radius)
        print(f"After inflation: {len(vox_idx)} (added {len(vox_idx) - original_count})")
    
    return vox_idx, origin

def indices_to_open3d_voxelgrid(indices, voxel_size, origin):
    """
    Convert voxel indices to Open3D VoxelGrid for visualization
    """
    if len(indices) == 0:
        return o3d.geometry.VoxelGrid()
    
    centers = voxel_indices_to_centers(indices, voxel_size, origin)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(centers)
    
    # Create bounds
    min_bound = origin - voxel_size
    max_bound = centers.max(axis=0) + voxel_size * 2
    print(f"[indices_to_o3d] min_bound: {min_bound}  |  max_bound: {max_bound}")
    
    vg = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd, voxel_size, min_bound=min_bound, max_bound=max_bound
    )
    
    return vg

def create_inflated_voxelgrid(inflated_indices, voxel_size, origin):
    """
    Create VoxelGrid from inflated indices (no color coding)
    """
    # return empty grid if no voxels
    if len(inflated_indices) == 0:
        return o3d.geometry.VoxelGrid()
    
    # convert indices to world coordinate centers
    centers = voxel_indices_to_centers(inflated_indices, voxel_size, origin)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(centers)
    
    # Create bounds
    min_bound = origin - voxel_size
    max_bound = centers.max(axis=0) + voxel_size * 2
    print(f"[create_inf_voxel] min_bound: {min_bound}  |  max_bound: {max_bound}")

    vg = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd, voxel_size, min_bound=min_bound, max_bound=max_bound
    )
    
    return vg

# Animate the map being generated as the camera moves

In [ ]:
def animate_camera_view_as_moves(depth_images, poses_Twc, cam_info, zoom=0.2):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Map Animation")
    
    K = np.array(cam_info.K).reshape(3, 3)
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    # Initialize with first frame's points before adding to visualizer
    first_pts = depth_to_world_points(depth_images[0], poses_Twc[0], fx, fy, cx, cy, stride=4)
   
    # add the non-empty geometry
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(first_pts)
    vis.add_geometry(pcd)
    
    state = {"index": 1, "total": len(poses_Twc)}  # Start from index 1
    
    def move_camera(vis):
        i = state['index']
        
        if i >= state["total"]:
            vis.register_animation_callback(None)
            print("Animation complete")
            return False
        
        Twc = poses_Twc[i]
        depth = depth_images[i]
        
        # Generate points for current frame
        pts = depth_to_world_points(depth, Twc, fx, fy, cx, cy, stride=4)
        
        if pts.size > 0:  # Only add if we have valid points
            # Append new points to running map
            old = np.asarray(pcd.points)
            pcd.points = o3d.utility.Vector3dVector(np.vstack([old, pts]))
            
            # Mark geometry as updated
            vis.update_geometry(pcd)
        
        print(f"[{i}] Total points in map: {len(pcd.points)}")
        
        # Update camera view based on current pose
        ctr = vis.get_view_control()
        
        # Extract camera axes
        forward = Twc[:3, 2] / np.linalg.norm(Twc[:3, 2]) # Z-axis in world
        up = Twc[:3, 1] / np.linalg.norm(Twc[:3, 1])      # Y-axis in world
        cam_center = Twc[:3, 3] # Translation vector
        
        # Set view parameters
        lookat = cam_center + forward
        ctr.set_front(-forward)
        ctr.set_up(-up)
        ctr.set_lookat(lookat)
        ctr.set_zoom(zoom)
        
        vis.poll_events()
        vis.update_renderer()
        
        state["index"] += 1
        return False
    
    vis.register_animation_callback(move_camera)
    vis.run()
    vis.destroy_window()

### Running script

In [8]:
cam_info, poses_frame_Twc, depth_frames = read_bag_file(bag_path, info_topic, pose_topic, depth_topic)

Reading  camera intrinsics...
Camera intrinsics: fx=226.380, fy=226.150, cx=173.647, cy=133.733
Reading poses...
Loaded 5695 poses.
Reading depth frames...
Loaded 1399 depth frames.


In [9]:
depth_images, poses_Twc = match_depth_pose(depth_frames, poses_frame_Twc)

Matched 1398 depth frames with poses.


In [10]:
pcd = o3d.geometry.PointCloud()
generate_point_cloud(pcd, depth_images, poses_Twc, K = np.array(cam_info.K).reshape(3, 3))

# save point cloud
# o3d.io.write_point_cloud("map_pointcloud.ply", pcd)

In [16]:
# Method 1
print("Create Voxel Map")
pts = apply_colorizaation(pcd) # better visualization
voxel_size = 0.01  # in meters
vg = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
o3d.io.write_voxel_grid(f"map_voxels_{voxel_size}.ply", vg)

Create Voxel Map


True

In [17]:
# Method 2
print("\nStep 1: Creating original voxel indices...")
vox_idx_original, origin = build_voxel_set_from_points(
    pts, voxel_size=VOXEL_SIZE, inflate_radius=0.0
)
print(f"Original occupied voxels: {len(vox_idx_original)}")


Step 1: Creating original voxel indices...
[min_p] [-4.17519998 -5.39259681 -0.13159429]  |  [origin] [-4.17524998 -5.39264681 -0.13164429]
idx: [[ 26  63  77]
 [ 26  62  78]
 [ 22  58  86]
 ...
 [ 64 108   8]
 [ 65 109  12]
 [ 74 109  12]]
 uniq_idx: [[  0  81  58]
 [  1  75  64]
 [  1  79  54]
 ...
 [155  86  39]
 [155  86  40]
 [155  87  38]]
Original occupied voxels: 215153


In [18]:
print("\nStep 2: Inflating voxels...")
# Use origin calcuated from creating original voxel indices
vox_idx_inflated, _ = build_voxel_set_from_points(
    pts, voxel_size=VOXEL_SIZE, origin=origin, inflate_radius=INFLATION_RADIUS
)


Step 2: Inflating voxels...
idx: [[ 26  63  77]
 [ 26  62  78]
 [ 22  58  86]
 ...
 [ 64 108   8]
 [ 65 109  12]
 [ 74 109  12]]
 uniq_idx: [[  0  81  58]
 [  1  75  64]
 [  1  79  54]
 ...
 [155  86  39]
 [155  86  40]
 [155  87  38]]
Original voxels: 215153
Inflation: 925 offset voxels in sphere of radius 7 voxels
After inflation: 778008 (added 562855)


In [22]:
print("\nStep 3: Creating visualizations...")
vg_original = indices_to_open3d_voxelgrid(vox_idx_original, VOXEL_SIZE, origin)
vg_inflated = create_inflated_voxelgrid(vox_idx_inflated, VOXEL_SIZE, origin)


Step 3: Creating visualizations...
[indices_to_o3d] min_bound: [-4.22524998 -5.44264681 -0.18164429]  |  max_bound: [3.69975002 0.23235319 8.84335571]
[create_inf_voxel] min_bound: [-4.22524998 -5.44264681 -0.18164429]  |  max_bound: [3.99975002 0.53235319 9.14335571]


In [ ]:
# Save
print("\nSaving voxel grids...")
o3d.io.write_voxel_grid("map_voxels_original.ply", vg_original)
o3d.io.write_voxel_grid("map_voxels_inflated.ply", vg_inflated)

In [ ]:
animate_camera_view_as_moves(depth_images, poses_Twc, cam_info)

In [ ]:
from tqdm import tqdm
import time

def depth_to_points(depth_img, fx, fy, cx, cy, max_depth=10.0):
    """
    Convert depth image to 3D points in camera frame.
    
    Args:
        depth_img: HxW depth image (meters)
        fx, fy: focal lengths in pixels
        cx, cy: principal point (image center) in pixels
        max_depth: maximum valid depth in meters
    
    Returns:
        Nx3 array of 3D points in camera coordinates
    """
    h, w = depth_img.shape
    
    # Create pixel coordinate grids
    v, u = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
    
    # Flatten to 1D arrays
    u = u.flatten()
    v = v.flatten()
    z = depth_img.flatten()
    
    # Filter valid depths
    valid_mask = (z > 0) & (z < max_depth)
    u = u[valid_mask]
    v = v[valid_mask]
    z = z[valid_mask]
    
    # Back-project to 3D using pinhole camera model
    # x = (u - cx) * z / fx
    # y = (v - cy) * z / fy
    x = (u - cx) * z / fx
    y = (v - cy) * z / fy
    
    # Stack into Nx3 array [x, y, z]
    points = np.stack([x, y, z], axis=1)
    
    return points


def transform_points(points, pose):
    """
    Transform points from camera frame to world frame using pose matrix.
    
    Args:
        points: Nx3 array of points in camera frame
        pose: 4x4 transformation matrix [R|t; 0 0 0 1]
              R is 3x3 rotation, t is 3x1 translation
    
    Returns:
        Nx3 array of points in world frame
    """
    if points.shape[0] == 0:
        return points
    
    # Convert to homogeneous coordinates (Nx4)
    points_homog = np.hstack([points, np.ones((points.shape[0], 1))])
    
    # Apply transformation: (4x4) @ (4xN) -> (4xN)
    points_world_homog = (pose @ points_homog.T).T
    
    # Convert back to 3D (drop homogeneous coordinate)
    points_world = points_world_homog[:, :3]
    
    return points_world


def animate_voxel_camera_view_as_moves(depth_data, poses, cam_info, voxel_size=0.05, inflate_radius=0.0, 
                                  frame_skip=10, max_frames=None):
    """
    Animate voxel map being built frame by frame
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Voxel Map Animation", width=1280, height=720)
    
    # Initialize empty voxel grid
    voxel_grid = o3d.geometry.VoxelGrid()
    vis.add_geometry(voxel_grid)
    
    # Add coordinate frame for reference
    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)
    vis.add_geometry(coord_frame)
    
    # Track accumulated voxel indices
    accumulated_voxels = []
    origin = None
    
    K = np.array(cam_info.K).reshape(3, 3)
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    
    n_frames = len(depth_data) if max_frames is None else min(max_frames, len(depth_data))
    
    for i in tqdm(range(0, n_frames, frame_skip), desc="Animating voxel map"):
        depth_img = depth_data[i]
        pose = poses[i]
        
        # Get points for this frame
        points_cam = depth_to_points(depth_img, fx=fx, fy=fy, cx=cx, cy=cy)
        if points_cam.shape[0] == 0:
            continue
        
        points_world = transform_points(points_cam, pose)
        
        # Convert to voxel indices
        vox_idx, frame_origin = points_to_voxel_indices(points_world, voxel_size, origin)
        if origin is None:
            origin = frame_origin
        
        # Accumulate indices
        if len(accumulated_voxels) == 0:
            accumulated_voxels = vox_idx
        else:
            accumulated_voxels = np.vstack([accumulated_voxels, vox_idx])
            accumulated_voxels = np.unique(accumulated_voxels, axis=0)
        
        # Apply inflation if requested
        if inflate_radius > 0:
            display_voxels = inflate_voxel_indices(accumulated_voxels, voxel_size, inflate_radius)
        else:
            display_voxels = accumulated_voxels
        
        # Convert to Open3D VoxelGrid
        new_voxel_grid = indices_to_open3d_voxelgrid(display_voxels, voxel_size, origin)
        
        # Update geometry
        voxel_grid.voxels = new_voxel_grid.voxels
        voxel_grid.voxel_size = new_voxel_grid.voxel_size
        voxel_grid.origin = new_voxel_grid.origin
        
        vis.update_geometry(voxel_grid)
        vis.poll_events()
        vis.update_renderer()
        
        time.sleep(0.01)  # Small delay for visualization
    
    vis.run()
    vis.destroy_window()



animate_voxel_camera_view_as_moves(depth_images, poses_Twc, cam_info, VOXEL_SIZE)

In [ ]:
import numpy as np
import open3d as o3d
import cv2
from pathlib import Path
import os

class VoxelMap:
    
    def __init__(self, voxel_size=0.05, max_weight=100):
        """
        Args:
            voxel_size: Size of each voxel cube (in meters)
            max_weight: Maximum weight for TSDF (prevents overflow)
        """
        self.voxel_size = voxel_size
        self.max_weight = max_weight
        
        # Create voxel grid
        self.voxel_grid = o3d.geometry.VoxelGrid()
        self.voxel_grid.voxel_size = voxel_size
        
        # Storage for voxel data (using dict for sparse representation)
        self.voxel_data = {}  # key: (x,y,z) voxel index -> value: count
        
    def world_to_voxel_index(self, points):
        """Convert world coordinates to voxel indices."""
        return np.floor(points / self.voxel_size).astype(np.int32)
    
    def add_points(self, points):
        """Add new points to the voxel map."""
        if points.size == 0:
            return
        
        # Convert to voxel indices
        voxel_indices = self.world_to_voxel_index(points)
        
        # Update occupancy counts
        for idx in voxel_indices:
            key = tuple(idx)
            if key in self.voxel_data:
                self.voxel_data[key] = min(self.voxel_data[key] + 1, self.max_weight)
            else:
                self.voxel_data[key] = 1
    
    def get_point_cloud(self, min_count=1):
        """
        Extract point cloud from voxel map.
        
        Args:
            min_count: Minimum observation count to include voxel
        """
        # Filter voxels by count
        valid_voxels = [k for k, v in self.voxel_data.items() if v >= min_count]
        
        if len(valid_voxels) == 0:
            return np.array([]).reshape(0, 3)
        
        # Convert voxel indices back to world coordinates (center of voxel)
        voxel_centers = np.array(valid_voxels) * self.voxel_size + self.voxel_size / 2
        return voxel_centers
    
    def get_voxel_grid(self):
        """Get Open3D VoxelGrid for visualization."""
        # Create point cloud from voxel centers
        points = self.get_point_cloud(min_count=1)
        
        if points.size == 0:
            return o3d.geometry.VoxelGrid()
        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        
        # Convert to voxel grid
        voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(
            pcd, voxel_size=self.voxel_size
        )
        return voxel_grid


def animate_voxel_map(depth_images, poses_Twc, cam_info, 
                      voxel_size=0.05, zoom=0.2,
                      save_video=True, output_path="voxel_map.mp4", 
                      fps=30, temp_dir="temp_frames"):
    """
    Animate voxel map construction as each frame arrives.
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Voxel Map Animation", width=1920, height=1080)
    
    K = np.array(cam_info.K).reshape(3, 3)
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    # Create temp directory for frames
    if save_video:
        Path(temp_dir).mkdir(exist_ok=True)
    
    # Initialize voxel map
    voxel_map = VoxelMap(voxel_size=voxel_size)
    
    # Process first frame to initialize
    first_pts = depth_to_world_points(depth_images[0], poses_Twc[0], fx, fy, cx, cy, stride=4)
    voxel_map.add_points(first_pts)
    
    # Get initial point cloud for visualization
    pcd = o3d.geometry.PointCloud()
    initial_points = voxel_map.get_point_cloud(min_count=1)
    pcd.points = o3d.utility.Vector3dVector(initial_points)
    vis.add_geometry(pcd)
    
    state = {"index": 1, "total": len(poses_Twc)}
    
    def update_frame(vis):
        i = state['index']
        
        if i >= state["total"]:
            vis.register_animation_callback(None)
            print("Animation complete")
            return False
        
        Twc = poses_Twc[i]
        depth = depth_images[i]
        
        #  Convert to world points
        pts = depth_to_world_points(depth, Twc, fx, fy, cx, cy, stride=4)
        
        # Add to voxel grid (update occupancy)
        if pts.size > 0:
            voxel_map.add_points(pts)
            
            # Update visualization
            voxel_points = voxel_map.get_point_cloud(min_count=1)
            pcd.points = o3d.utility.Vector3dVector(voxel_points)
            vis.update_geometry(pcd)
        
        print(f"[{i}/{state['total']}] Voxels: {len(voxel_map.voxel_data)}, "
              f"Points: {len(pcd.points)}")
        
        # Update camera view
        ctr = vis.get_view_control()
        forward = Twc[:3, 2] / np.linalg.norm(Twc[:3, 2])
        up = Twc[:3, 1] / np.linalg.norm(Twc[:3, 1])
        cam_center = Twc[:3, 3]
        
        lookat = cam_center + forward
        ctr.set_front(-forward)
        ctr.set_up(-up)
        ctr.set_lookat(lookat)
        ctr.set_zoom(zoom)
        
        # Render a new frame
        vis.poll_events()
        vis.update_renderer()
        
        # Save frame
        if save_video:
            frame_path = f"{temp_dir}/frame_{i:05d}.png"
            vis.capture_screen_image(frame_path, do_render=True)
        
        state["index"] += 1
        return False
    
    vis.register_animation_callback(update_frame)
    vis.run()
    vis.destroy_window()
    
    if save_video:
        compile_frames_to_video(temp_dir, output_path, fps)
    
    return voxel_map


def compile_frames_to_video(frame_dir, output_path, fps=30):
    """Compile PNG frames into a video."""
    frame_files = sorted([f for f in os.listdir(frame_dir) if f.endswith('.png')])
    
    if len(frame_files) == 0:
        print("No frames found!")
        return
    
    first_frame = cv2.imread(os.path.join(frame_dir, frame_files[0]))
    height, width = first_frame.shape[:2]
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Compiling {len(frame_files)} frames into video...")
    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(frame_dir, frame_file))
        out.write(frame)
    
    out.release()
    print(f"Video saved to {output_path}")

# Simple voxel grid
voxel_map = animate_voxel_map(
    depth_images, poses_Twc, cam_info,
    voxel_size=0.05,  # 5cm voxels
    zoom=0.2,
    save_video=True,
    output_path="voxel_animation.mp4"
)

[1/1398] Voxels: 1300, Points: 1300
[2/1398] Voxels: 1722, Points: 1722
[3/1398] Voxels: 2147, Points: 2147
[4/1398] Voxels: 2394, Points: 2394
[5/1398] Voxels: 2643, Points: 2643
[6/1398] Voxels: 2980, Points: 2980
[7/1398] Voxels: 3207, Points: 3207
[8/1398] Voxels: 3396, Points: 3396
[9/1398] Voxels: 3627, Points: 3627
[10/1398] Voxels: 3931, Points: 3931
[11/1398] Voxels: 4197, Points: 4197
[12/1398] Voxels: 4350, Points: 4350
[13/1398] Voxels: 4581, Points: 4581
[14/1398] Voxels: 4718, Points: 4718
[15/1398] Voxels: 4838, Points: 4838
[16/1398] Voxels: 5017, Points: 5017
[17/1398] Voxels: 5089, Points: 5089
[18/1398] Voxels: 5178, Points: 5178
[19/1398] Voxels: 5381, Points: 5381
[20/1398] Voxels: 5621, Points: 5621
[21/1398] Voxels: 5689, Points: 5689
[22/1398] Voxels: 5962, Points: 5962
[23/1398] Voxels: 6165, Points: 6165
[24/1398] Voxels: 6321, Points: 6321
[25/1398] Voxels: 6426, Points: 6426
[26/1398] Voxels: 6562, Points: 6562
[27/1398] Voxels: 6754, Points: 6754
[28/1398] 

# Advanced version with TSDF

In [20]:
import os 

class TSDFVoxelMap:
    """TSDF-based voxel map for better reconstruction."""
    
    def __init__(self, voxel_size=0.05, sdf_trunc=0.04):
        """
        Args:
            voxel_size: Size of each voxel
            sdf_trunc: Truncation distance for TSDF
        """
        self.voxel_size = voxel_size
        self.sdf_trunc = sdf_trunc
        
        # Use Open3D's TSDF volume
        self.volume = o3d.pipelines.integration.ScalableTSDFVolume(
            voxel_length=voxel_size,
            sdf_trunc=sdf_trunc,
            color_type=o3d.pipelines.integration.TSDFVolumeColorType.NoColor
        )
    
    def integrate_frame(self, depth, Twc, fx, fy, cx, cy):
        """
        Integrate a depth frame into the TSDF volume.
        
        Args:
            depth: Depth image (H x W)
            Twc: Camera-to-world pose (4x4)
            fx, fy, cx, cy: Camera intrinsics
        """
        # Convert depth to Open3D format
        depth_o3d = o3d.geometry.Image(depth.astype(np.float32))
        
        # Create intrinsic object
        height, width = depth.shape
        intrinsic = o3d.camera.PinholeCameraIntrinsic(
            width, height, fx, fy, cx, cy
        )
        
        # Create RGBD image (depth only)
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            o3d.geometry.Image(np.zeros((height, width, 3), dtype=np.uint8)),
            depth_o3d,
            depth_scale=1.0,
            depth_trunc=10.0,
            convert_rgb_to_intensity=False
        )
        
        # Integrate into volume
        # Note: TSDF expects extrinsic (world-to-camera), so we invert Twc
        Tcw = np.linalg.inv(Twc)
        self.volume.integrate(rgbd, intrinsic, Tcw)
    
    def extract_point_cloud(self):
        """Extract point cloud from TSDF volume."""
        return self.volume.extract_point_cloud()
    
    def extract_mesh(self):
        """Extract triangle mesh from TSDF volume."""
        return self.volume.extract_triangle_mesh()


def animate_tsdf_map(depth_images, poses_Twc, cam_info,
                     voxel_size=0.05, zoom=0.2,
                     save_video=True, output_path="tsdf_map.mp4",
                     fps=30, temp_dir="temp_frames"):
    """Animate TSDF voxel map construction."""
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="TSDF Map Animation", width=1920, height=1080)
    
    K = np.array(cam_info.K).reshape(3, 3)
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    if save_video:
        Path(temp_dir).mkdir(exist_ok=True)
    
    # Initialize TSDF volume
    tsdf_map = TSDFVoxelMap(voxel_size=voxel_size)
    
    # Integrate first frame
    tsdf_map.integrate_frame(depth_images[0], poses_Twc[0], fx, fy, cx, cy)
    
    # Extract and add to visualizer
    pcd = tsdf_map.extract_point_cloud()
    vis.add_geometry(pcd)
    
    state = {"index": 1, "total": len(poses_Twc)}
    
    def update_frame(vis):
        i = state['index']
        
        if i >= state["total"]:
            vis.register_animation_callback(None)
            print("Animation complete")
            return False
        
        Twc = poses_Twc[i]
        depth = depth_images[i]
        
        # Integrate new frame into TSDF
        tsdf_map.integrate_frame(depth, Twc, fx, fy, cx, cy)
        
        # Extract updated point cloud
        new_pcd = tsdf_map.extract_point_cloud()
        pcd.points = new_pcd.points
        vis.update_geometry(pcd)
        
        print(f"[{i}/{state['total']}] Points: {len(pcd.points)}")
        
        # Update camera view
        ctr = vis.get_view_control()
        forward = Twc[:3, 2] / np.linalg.norm(Twc[:3, 2])
        up = Twc[:3, 1] / np.linalg.norm(Twc[:3, 1])
        cam_center = Twc[:3, 3]
        
        lookat = cam_center + forward
        ctr.set_front(-forward)
        ctr.set_up(-up)
        ctr.set_lookat(lookat)
        ctr.set_zoom(zoom)
        
        vis.poll_events()
        vis.update_renderer()
        
        if save_video:
            vis.capture_screen_image(f"{temp_dir}/frame_{i:05d}.png", do_render=True)
        
        state["index"] += 1
        return False
    
    vis.register_animation_callback(update_frame)
    vis.run()
    vis.destroy_window()
    
    if save_video:
        compile_frames_to_video(temp_dir, output_path, fps)
    
    return tsdf_map


# TSDF-based (better quality)
tsdf_map = animate_tsdf_map(
    depth_images, poses_Twc, cam_info,
    voxel_size=0.05,
    zoom=0.2,
    save_video=True,
    output_path="tsdf_animation.mp4"
)
# Extract final mesh
final_mesh = tsdf_map.extract_mesh()
o3d.io.write_triangle_mesh("final_mesh.ply", final_mesh)

[1/1398] Points: 779
[2/1398] Points: 999
[3/1398] Points: 1289
[4/1398] Points: 1390
[5/1398] Points: 1470
[6/1398] Points: 1594
[7/1398] Points: 1658
[8/1398] Points: 1695
[9/1398] Points: 1745
[10/1398] Points: 1897
[11/1398] Points: 1927
[12/1398] Points: 1980
[13/1398] Points: 2062
[14/1398] Points: 2128
[15/1398] Points: 2174
[16/1398] Points: 2211
[17/1398] Points: 2255
[18/1398] Points: 2299
[19/1398] Points: 2320
[20/1398] Points: 2323
[21/1398] Points: 2365
[22/1398] Points: 2433
[23/1398] Points: 2472
[24/1398] Points: 2550
[25/1398] Points: 2616
[26/1398] Points: 2615
[27/1398] Points: 2681
[28/1398] Points: 2703
[29/1398] Points: 2742
[30/1398] Points: 2808
[31/1398] Points: 2831
[32/1398] Points: 2872
[33/1398] Points: 2877
[34/1398] Points: 2898
[35/1398] Points: 2893
[36/1398] Points: 2901
[37/1398] Points: 2919
[38/1398] Points: 2918
[39/1398] Points: 2954
[40/1398] Points: 2979
[41/1398] Points: 2996
[42/1398] Points: 3019
[43/1398] Points: 3056
[44/1398] Points: 3072

True